# 주의!

train 파일과 test(infer) 파일 **모두**에 아래의 wandb 함수들을 박아두는 게 안전합니다.  
run 이름을 바꿔서 구분하면 되니까 **일단 백업합시다!!**

## wandb.init()

먼저 wandb를 설치합니다.

In [12]:
!pip install wandb

init() 함수로 프로젝트를 생성합니다.  
'config' parameter로 hyperparameter 값이 정리된 dictionary를 init()에 넘겨주고, 편리한 사용을 위해 이를 로컬 변수 CFG에 저장합니다.   
백업을 위해 프로젝트의 루트 디렉토리를 wandb에게 알려주려면 'settings' parameter에 아래와 같은 형식으로 넘겨줍니다.

In [13]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

import random
import numpy as np
import tensorflow as tf

root_dir = '/content/test-project'
wandb.init(
    project="wandb-tensorflow-tutorial",
    name="baseline",
    config= {
      "layer_1": 512,
      "activation_1": "relu",
      "dropout": random.uniform(0.01, 0.80),
      "layer_2": 10,
      "activation_2": "softmax",
      "optimizer": "sgd",
      "loss": "sparse_categorical_crossentropy",
      "metric": "accuracy",
      "epoch": 8,
      "batch_size": 256
    },

    # to save all codes in current dir and all subdirs as an artifact
    # https://docs.wandb.ai/guides/app/features/panels/code
    settings=wandb.Settings(code_dir=root_dir),
)

CFG = wandb.config

wandb: Currently logged in as: js1044k (rokafai). Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [14]:
import os

if not os.path.exists(root_dir):
    os.makedirs(root_dir)

## wandb callbacks - WandbMetricsLogger

tensorflow에서는 pytorch와 달리 주로 model.fit()을 이용해 학습시키므로, fit()의 parameter로 wandb의 callback 함수들을 넘겨줘야 logging 등의 작업이 가능합니다.  

Callback 함수들에 대한 자세한 설명은 [여기로](https://https://docs.wandb.ai/ref/python/integrations/keras/)

In [15]:
# get the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, y_train = x_train[::5], y_train[::5]
x_test, y_test = x_test[::20], y_test[::20]
labels = [str(digit) for digit in range(np.max(y_train) + 1)]

# build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(CFG.layer_1, activation=CFG.activation_1),
    tf.keras.layers.Dropout(CFG.dropout),
    tf.keras.layers.Dense(CFG.layer_2, activation=CFG.activation_2)
    ])

# compile the model
model.compile(optimizer=CFG.optimizer,
              loss=CFG.loss,
              metrics=[CFG.metric]
              )

# save best model
checkpoint_filepath = f'{root_dir}/model/best_model.hdf5'
mc = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_filepath,
  save_weights_only=True,
  monitor='val_loss',
  mode='min',
  save_best_only=True)

WandbMetricsLogger는 'log_freq' 주기의 batch마다 metric을 기록합니다.   
batch 단위가 싫으면 비워두세요. 그럼 1 epoch 단위로 기록합니다.

In [16]:
history = model.fit(x=x_train, y=y_train,
                    epochs=CFG.epoch,
                    batch_size=CFG.batch_size,
                    validation_data=(x_test, y_test),
                    callbacks=[
                      mc,
                      WandbMetricsLogger()
                    ])

Epoch 1/8
47/47 [==============================] - 2s 23ms/step - loss: 2.1136 - accuracy: 0.3085 - val_loss: 1.8565 - val_accuracy: 0.5500
Epoch 2/8
47/47 [==============================] - 2s 33ms/step - loss: 1.6773 - accuracy: 0.6271 - val_loss: 1.4886 - val_accuracy: 0.7280
Epoch 3/8
47/47 [==============================] - 1s 26ms/step - loss: 1.3655 - accuracy: 0.7288 - val_loss: 1.2250 - val_accuracy: 0.7760
Epoch 4/8
47/47 [==============================] - 1s 26ms/step - loss: 1.1439 - accuracy: 0.7713 - val_loss: 1.0414 - val_accuracy: 0.7940
Epoch 5/8
47/47 [==============================] - 1s 26ms/step - loss: 0.9857 - accuracy: 0.7969 - val_loss: 0.9123 - val_accuracy: 0.8000
Epoch 6/8
47/47 [==============================] - 1s 26ms/step - loss: 0.8767 - accuracy: 0.8152 - val_loss: 0.8186 - val_accuracy: 0.8100
Epoch 7/8
47/47 [==============================] - 1s 27ms/step - loss: 0.7925 - accuracy: 0.8257 - val_loss: 0.7500 - val_accuracy: 0.8180
Epoch 8/8
47/47 [===

## wandb.save()

실행 과정에서 생성되는 파일을 저장하고 싶다면 **wandb.finish() 직전에** wandb.save()를 실행해주세요.  
저장된 파일들은 해당 run의 **'files':** 항목에서 확인할 수 있습니다.  

형식: wandb.save(저장할 파일명 또는 경로, base_path=루트 디렉토리)

In [17]:
import os
import pandas as pd

if not os.path.exists(f'{root_dir}/result'):
    os.makedirs(f'{root_dir}/result')

submission = pd.read_csv(f'/content/sample_data/mnist_test.csv')

submission_path = f'{root_dir}/result/submission.csv'
submission.to_csv(submission_path)

model_path = checkpoint_filepath

In [18]:
wandb.save(model_path, base_path=root_dir)
wandb.save(submission_path, base_path=root_dir)

['/content/wandb/run-20231125_101101-a8d2zxb0/files/result/submission.csv']

파일 경로는 엉망이어도 되니 일단 전부 저장해야 한다면...

In [19]:
# set wandb to save all codes, weights, and results
wandb.save("*.py")
wandb.save("*.ipynb")

wandb.save("*.pt")
wandb.save("*.pth")
wandb.save("*.hdf5")

wandb.save("*.csv")

wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.
wandb: WARNING Symlinked 0 file into the W&B run directory, call wandb.save again to sync new files.


[]

## wandb.finish()

코드 실행이 끝나기 전 wandb.finish()를 호출해야 wandb에서도 제대로 실행이 마무리된 걸로 처리됩니다.  

In [20]:
wandb.finish()

epoch/accuracy,▁▅▇▇████
epoch/epoch,▁▂▃▄▅▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▃▂▂▁▁
epoch/val_accuracy,▁▆▇▇▇███
epoch/val_loss,█▆▄▃▂▂▁▁
epoch/accuracy,0.833
epoch/epoch,7
epoch/learning_rate,0.01
epoch/loss,0.73416
epoch/val_accuracy,0.826
